# 動的モード分解(Dynamic Mode Decomposition)

この章では、動的モード分解(Dynamic Mode Decomposition, DMD)を紹介する。

DMDは、時空間データから、系のダイナミクスを解析するための手法である。
元々、流体力学の分野で提唱された比較的新しい手法で、
その後、複雑系のダイナミクスを解析するための手法として、様々な分野で応用されている。

本章で使用するパッケージをインストールしておこう (1回実行すればよいため、コメントアウトしておいた)

In [ ]:
# using Pkg 
# Pkg.add("MAT")
# Pkg.add("Arpack")
# Pkg.add("KrylovKit")
# Pkg.add("Plots")

※一般論は別のノートブックにまとめているので、省略した。

## 具体例: Kármán渦

DMDの有名な教科書(Kutzら)にある、流体力学の例を紹介する。

- 教科書: Data-Driven Modeling & Scientific Computation: Methods for Complex Systems & Big Data, J. Nathan Kutz, Cambridge University Press, 2013.
- URL: http://dmdbook.com/

こちらのページからは、DataとMATLABのコードがダウンロードできる。
ここで提供されているデータは、Kármán(カルマン)渦と呼ばれる、円筒形の物体の周りに出来る流体の渦のシミュレーション結果である。
カルマン渦が見られる状況としては、例えば水や雲の流れといった自然現象や建築物(煙突)や潜水艦の潜望鏡など多数の例がある。


データの中身は、200×450のメッシュ領域における、199×449の速度場(流れ)が、時間方向に151ステップ記録されている。
1ステップの時間間隔は、0.2で、$t=0 \sim 30.0$のデータがある。
このデータは、199×449=89351次元のベクトルが151個ある状態とみなせる。


このデータに対してDMDを適用してみよう。カレントディレクトリに、`DATA`をダウンロード・展開済と仮定して、まずはデータを可視化してみよう。

実装においては、グローバルスコープで行列や行列演算を定義すると、実行時間が膨大になってしまうことをさけるため、
関数の内部で行列の確保や演算を行うようにしているため、少々見づらいかもしれない。

In [ ]:
using Plots
using MAT
file = matopen("DATA/FLUIDS/CYLINDER_ALL.mat")

u_all = read(file, "UALL")
v_all = read(file, "VALL")
vorticity_all = read(file, "VORTALL")
println("size(u_all) = ", size(u_all), " size(v_all) = ", size(v_all), " size(vorticity_all) = ", size(vorticity_all))

In [ ]:
nrow = 199
ncol = 449

function make_vec_to_mat(vec, nrow, ncol, order="r")
    mat = zeros(Float64, nrow, ncol)
    if order == "r"
        for i = 1:length(vec)
            mat[i] = vec[i]
        end
    else #update matrix by vec in colum order
        for i = 1:length(vec)
            idx_i = div(i, ncol) + 1
            idx_j = i - (idx_i - 1) * ncol
            mat[idx_i, idx_j] = vec[i]
        end
    end
    return mat
end


In [ ]:
anim = @animate for i = 1:size(u_all)[2]
    mat = make_vec_to_mat(u_all[:,i], nrow, ncol)
    tnow = round((i-1) * 0.2, digits=1)
    heatmap(mat, aspect_ratio=:equal, color=:viridis, c=:balance, clim=(-1, 1), size=(400, 200), title="t = $tnow")
end
gif(anim, "dmd_data_u.gif", fps = 20)

In [ ]:
anim = @animate for i = 1:size(v_all)[2]
    mat = make_vec_to_mat(v_all[:,i], nrow, ncol)
    tnow = round((i-1) * 0.2, digits=1)
    heatmap(mat, aspect_ratio=:equal, color=:viridis, c=:balance, clim=(-1, 1), size=(400, 200),title="t = $tnow")
end
gif(anim, "dmd_data_v.gif", fps = 20)

In [ ]:
anim = @animate for i = 1:size(vorticity_all)[2]
    mat = make_vec_to_mat(vorticity_all[:,i], nrow, ncol)
    tnow = round((i-1) * 0.2, digits=1)
    heatmap(mat, aspect_ratio=:equal, color=:viridis, c=:balance, clim=(-1, 1), size=(400, 200),title="t = $tnow")
end
gif(anim, "dmd_data_vorticity.gif", fps = 20)

:::{margin}
JuliaのPlotsでアニメ作るの簡単でびっくり
:::
データを可視化出来た。では実際に、89351×151のデータについて、DMDを適用してみる。


In [ ]:
function dmd(X, Y, r;verbose=false)
    # 1.2. truncated SVD using Arpack.jl  
    Z = svds(X; nsv=r)[1]
    U_r, S_r, Vt_r = Z.U, Z.S, Z.Vt

    # 3. tilde(A)
    S_r_inv = diagm( 1.0 ./ S_r)
    Z = BLAS.gemm('T', 'N', 1.0, Vt_r, S_r_inv)
    YZ = BLAS.gemm('N','N',1.0, Y, Z)
    Atilde = BLAS.gemm('T', 'N', 1.0, U_r, YZ)

    # reconstruction of Y
    # method-A: multiplying Atilde k times
    Y_latent = zeros(Float64, r, size(Y)[2])
    x1 = X[:,1]
    x1_r = BLAS.gemv('T', 1.0, U_r, x1)
    x_k = zeros(Float64,r)
    x_new = zeros(Float64,r) .+ x1_r
    for k = 1:size(Y)[2]
        x_k .= x_new
        BLAS.gemv!('N', 1.0, Atilde, x_k, 0.0, x_new)
        Y_latent[:,k] .= x_new
    end
    Yapprox = BLAS.gemm('N', 'N', 1.0, U_r, Y_latent)
    println("norm(Y-Yapprox,Inf) = ", norm(Y - Yapprox,Inf), " Fro. ", norm(Y - Yapprox,2))

    # method-B: adding DMD modes (verbose=trueにして、Aと一致するかチェックしておこう)
    if verbose
        # 4. eigen(tilde(A))
        Lambda, W = eigen(Atilde)

        # 5. DMD modes
        Phi = X * Vt_r' * S_r_inv * W
        Phi_pinv = pinv(Phi)

        # Y approximation
        Yapprox_B = zeros(Float64, size(Y))
        Lambda, W = eigen(Atilde)
        z = Phi_pinv * X[:,1] 
        for k = 1:size(Y)[2]  
            for i = 1:r
                tmp = Lambda[i]^(k) * z[i] * Phi[:,i]
                Yapprox_B[:,k] .+= real.(tmp)
            end
        end
        println("norm(Y-Yapprox,Inf) = ", norm(Y - Yapprox_B,Inf), " Fro. ", norm(Y - Yapprox_B,2))
    end
    return Yapprox
end

function main(target)

    dim = nrow*ncol
    num_t = size(target)[2]

    X = zeros(Float64, dim, num_t-1)
    Y = zeros(Float64, dim, num_t-1)

    for i = 1:num_t-1
        vec = target[:,i]
        X[:,i] .= vec
        if i > 1
        Y[:,i-1] .= vec
        end
    end
    Y[:,end] .= target[:,end]
    println("fullrank $(rank(X))")
    ## DMD
    trank = 15
    Yapprox = dmd(X, Y, trank)

    return Y, Yapprox
end

Y, Yapprox = main(u_all)


In [ ]:
anim = @animate for i = 1:size(Y)[2]
    mat = make_vec_to_mat(Y[:,i], nrow, ncol)
    mat_approx = make_vec_to_mat(Yapprox[:,i], nrow, ncol)
    hm1 = heatmap(mat, aspect_ratio=:equal, color=:viridis, c=:balance, clim=(-1, 1), size=(400, 200),title="Exact")
    hm2 = heatmap(mat_approx, aspect_ratio=:equal, color=:viridis, c=:balance, clim=(-1, 1), size=(400, 200),title="DMD")    
    diff = mat-mat_approx
    hm3 = heatmap(diff, aspect_ratio=:equal, color=:viridis, c=:balance, clim=(-1.e-2, 1.e-2), size=(400, 400),title="Diff.:=Exact-DMD")
    plot(hm1, hm2, hm3, layout=(3,1), size=(400, 800))
end
gif(anim, "dmd_data_compare.gif", fps = 20)


上から、元データ、DMDによる近似($r=15 < 150$)、両者の差分をプロットした。  
少なくとも目に見えて分かるような差分はなく、データの再構成がうまくいっていることがわかる。